# Data Acquisition via API Calls (NewsAPI)

---

## Phase 1: Setup and Security

### Install Dependencies

In [16]:
# Install the necessary library for loading environment variables (like API keys)
!pip install python-dotenv

print("Dependencies installed successfully.")


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Dependencies installed successfully.


### Import Modules
Import all necessary Python libraries. Note that `requests` is the primary tool for submitting HTTP requests.

In [17]:
import numpy as np
import pandas as pd
import os
import requests
import json
import dotenv
from datetime import datetime

print("Modules imported.")

Modules imported.


### Load Environment Variables
This loads your `.env` file, which is crucial for securely handling your NewsAPI key without exposing it in the notebook.

In [18]:
# Load the .env file to access environment variables (e.g., your API Key)
dotenv.load_dotenv()

print("Environment variables loaded.")

Environment variables loaded.


### Retrieve API Key
We access the stored API key using os.getenv().

In [19]:
# Get the News API key from the environment variables
newskey = os.getenv('newskey')

# We won't print the key, but we confirm it's loaded (e.g., by checking its length)
if newskey:
    print("News API key successfully retrieved.")
else:
    print("ERROR: News API key not found. Check your .env file.")

News API key successfully retrieved.


---

## Phase 2: Building the API Request Components
An HTTP request requires four main components: URL root, endpoint, headers (for security/metadata), and parameters (the query).

### Build User Agent Header
Many APIs require a User-agent to identify the client application. We use httpbin.org to dynamically grab a standard User-agent string.

In [20]:
# 1. Get a standard User-agent string
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

# 2. Build the full headers dictionary
# The API key is sent via the 'X-Api-Key' header, a common secure method.
headers = {'User-agent': useragent,
           'X-Api-Key': newskey}

print(f"User-agent: {useragent}")
print("Headers dictionary created, containing the API key.")

User-agent: python-requests/2.32.4
Headers dictionary created, containing the API key.


### Define URL Root and Endpoint
This defines the fixed address for the NewsAPI service we want to use.

In [21]:
# Define the fixed URL parts
root = 'https://newsapi.org'
endpoint = '/v2/everything' # This endpoint searches all articles

print(f"API Base URL: {root + endpoint}")

API Base URL: https://newsapi.org/v2/everything


### Define Query Parameters
Parameters are used to customize the search (e.g., topic, language, date).

In [22]:
# Define the search parameters as a Python dictionary
params = {'q': '"tallest mountain"',  # Topic to search for (using quotes for exact phrase)
         'searchIn': 'content',      # Search within the article content
         'language': 'en',           # Restrict to English articles
         'pageSize': 100}            # Request up to 100 articles

print("Query parameters defined:")
print(params)

Query parameters defined:
{'q': '"tallest mountain"', 'searchIn': 'content', 'language': 'en', 'pageSize': 100}


---

## Phase 3: API Execution and Parsing

### Submit the GET Request
This cell sends the request and checks the response status. A `<Response [200]>` indicates success.

In [23]:
# Combine the components and submit the GET request
r = requests.get(root + endpoint,
                headers = headers,
                params = params)

# Display the response object (it should show <Response [200]>)
print(f"Request submitted. Status: {r}")

Request submitted. Status: <Response [200]>


### Parse the JSON Response
The response (`r.text`) is a single string containing the JSON data. We use `json.loads()` to convert this string into a usable Python dictionary.

In [24]:
# Convert the JSON response string into a Python dictionary
myjson = json.loads(r.text)

print(f"JSON response converted to a Python dictionary (Type: {type(myjson)})")
print("Top-level keys in the response:")
print(list(myjson.keys()))

JSON response converted to a Python dictionary (Type: <class 'dict'>)
Top-level keys in the response:
['status', 'totalResults', 'articles']


### View Raw JSON Data Structure (Optional)
This cell is often left commented out to avoid printing a massive wall of text but serves as a way to inspect the data structure.

In [25]:
# Uncomment this line to inspect the full structure of the JSON dictionary
# print(json.dumps(myjson, indent=4))

### Normalize JSON to DataFrame
The key data is nested under the articles key. Pandas’ json_normalize() function flattens this nested data into a clean, tabular DataFrame.

In [26]:
# Use json_normalize to extract the list of article dictionaries ('articles')
news_df = pd.json_normalize(myjson, record_path = ['articles'])

print(f"DataFrame created with {len(news_df)} articles.")
print("First 5 rows of the DataFrame:")
display(news_df.head())

DataFrame created with 21 articles.
First 5 rows of the DataFrame:


,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Lydia Mansel,This Historic Train Climbs the Tallest Mountai...,Riding the Mount Washington Cog Railway is one...,https://www.travelandleisure.com/mount-washing...,https://s.yimg.com/ny/api/res/1.2/JmSXRiaGiWkv...,2025-09-20T16:00:00Z,Key Points\r\n<ul><li>The Mount Washington Cog...,None,Travel+Leisure
1,Express Web Desk,"Polish skier becomes first to climb, ski down ...","In 2018, the Polish climber was the first pers...",https://indianexpress.com/article/world/polish...,https://images.indianexpress.com/2025/09/polan...,2025-09-28T04:50:59Z,Polish skier Andrzej Bargiel made history this...,None,The Indian Express
2,Georgie English,CLEAREST EVER SIGNS OF LIFE ON MARS...,NASA has revealed the clearest signs of life o...,https://www.the-sun.com/tech/15159152/nasa-mar...,https://www.the-sun.com/wp-content/uploads/sit...,2025-09-10T17:34:37Z,NASA has revealed the clearest signs of life o...,None,The-sun.com
3,ABC News,"Sudan landslide claims 1,000 lives, village 'c...",The Sudan Liberation Movement/Army is appealin...,https://www.abc.net.au/news/2025-09-02/sudan-l...,https://live-production.wcms.abc-cdn.net.au/73...,2025-09-02T07:52:51Z,"At least 1,000 people were killed in a landsli...",abc-news-au,ABC News (AU)
4,"sascha.pare@futurenet.com (Sascha Pare) , Sasc...",The geology that holds up the Himalayas is not...,A 100-year-old theory explaining how Asia can ...,https://www.livescience.com/planet-earth/geolo...,https://cdn.mos.cms.futurecdn.net/KMwyEqed8eMT...,2025-08-30T15:50:00Z,Scientists may have just toppled a 100-year-ol...,None,Live Science


---

## Phase 4: Data Analysis and Export

### Clean and Prepare for Export
Perform a final step to ensure the data is properly formatted before saving.

In [27]:
# Clean up the publishedAt column (convert to datetime)
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

# Select a final set of columns for the CSV
final_df = news_df[['publishedAt', 'title', 'description', 'url', 'source.name', 'author']].copy()

print("DataFrame prepared for export.")

DataFrame prepared for export.


### Export to CSV
This is the final Load (L) phase of the data acquisition process.

In [28]:
# Define a filename based on the current date for organization
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'news_articles_{timestamp}.csv'

# Export the final clean DataFrame to a CSV file
final_df.to_csv(output_filename, index=False)

print(f"Successfully exported {len(final_df)} articles to {output_filename}")

Successfully exported 21 articles to news_articles_20250930_020114.csv


---

## Function Definition (Optional Consolidation)
This final cell provides the consolidation of all steps into a single, reusable function—demonstrating how a script would execute the entire API call process.

In [29]:
def grab_latest_articles():
    """
    Consolidates all steps: builds request, calls API, parses JSON, and returns a DataFrame.
    """
    
    # Prompt the user
    topic = input("Please enter your topic of interest: ")
    
    # Build our Headers
    newskey = os.getenv('newskey')
    r = requests.get('https://httpbin.org/user-agent')
    useragent = json.loads(r.text)['user-agent']
    headers = {'User-agent': useragent,
               'X-Api-Key': newskey}

    # Build our URL and Parameters
    root = 'https://newsapi.org'
    endpoint = '/v2/everything'
    params = {'q': topic,
              'searchIn': 'content',
              'language': 'en',
              'pageSize': 100}
    
    # Submit our Request
    r = requests.get(root + endpoint,
                headers = headers,
                params = params)
    
    # Create and return the pandas dataframe
    myjson = json.loads(r.text)
    news_df = pd.json_normalize(myjson, record_path = ['articles'])
    
    return news_df

In [30]:
grab_latest_articles()

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,GMA Team,MTV VMAs 2025: See the full winners list,Lady Gaga was the most nominated artist headin...,https://abcnews.go.com/GMA/Culture/mtv-vmas-20...,https://i.abcnewsfe.com/a/48518824-6426-45b8-b...,2025-09-08T03:40:40Z,The 2025 MTV VMAs featured some of music's big...,abc-news,ABC News
1,Matthew Strauss,The Weeknd Announces 2026 Tour Dates,The After Hours Til Dawn Stadium Tour continue...,https://pitchfork.com/news/the-weeknd-announce...,https://media.pitchfork.com/photos/68b9be9b0e7...,2025-09-04T16:32:31Z,If you thought the Weeknds After Hours Til Daw...,None,Pitchfork
2,Matthew Strauss,Cardi B to Replace the Weeknd at Global Citize...,The Canadian musician is pulling out of the Ne...,https://pitchfork.com/news/cardi-b-to-replace-...,https://media.pitchfork.com/photos/68d19331861...,2025-09-22T18:40:10Z,The 2025 Global Citizen Festival takes place t...,None,Pitchfork
3,info@hypebeast.com (Hypebeast),The Weeknd Announces 2026 Extension of “After ...,SummaryThe Weeknd announced an extension of hi...,https://hypebeast.com/2025/9/the-weeknd-announ...,https://image-cdn.hypb.st/https%3A%2F%2Fhypebe...,2025-09-05T06:45:31Z,Summary\r\n<ul><li>The Weeknd announced an ext...,None,HYPEBEAST
4,Walden Green,MTV VMAs 2025 Winners: See the Full List Here,"Who’ll win Video of the Year, Artist of the Ye...",https://pitchfork.com/news/mtv-vmas-2025-winne...,https://media.pitchfork.com/photos/68be02ec6ee...,2025-09-07T22:53:05Z,MTVs 2025 Video Music Awards are taking place ...,None,Pitchfork
...,...,...,...,...,...,...,...,...,...
92,Gregory Ellwood,"Lady Gaga, Ariana Grande & Sabrina Carpenter T...",For the first time — and perhaps a sign of the...,https://theplaylist.net/lady-gaga-ariana-grand...,https://cdn.theplaylist.net/wp-content/uploads...,2025-09-08T04:22:23Z,For the first time and perhaps a sign of the t...,None,Theplaylist.net
93,Aqua Boogie,Cardi B Steps In To Headline 2025 Global Citiz...,Cardi B is taking over the Global Citizen Fest...,https://www.rap-up.com/article/cardi-b-headlin...,https://www.rap-up.com/article/media_16999e5eb...,2025-09-22T19:53:43Z,Key Takeaways:\r\n<ul><li>Cardi B is set to pe...,None,Rap-Up.com
94,Malcolm Trapp,The Weeknd Plots 2026 Dates For “After Hours T...,Playboi Carti will rejoin The Weeknd next summ...,https://www.rap-up.com/article/the-weeknd-afte...,https://www.rap-up.com/article/media_1a63e0063...,2025-09-04T17:53:45Z,Key Takeaways:\r\n<ul><li>The Weeknds After Ho...,None,Rap-Up.com
95,saamaan1@yahoo.com (Master),Various Artists – Now That’s What I Call Music...,Tracklist:01 - Gracie Abrams - That’s So True0...,https://post.rlsbb.cc/various-artists-now-that...,None,2025-09-01T12:30:21Z,Tracklist:01 - Gracie Abrams - Thats So True02...,None,Rlsbb.cc
